### Agent ReAct Pattern

In [26]:
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch

from dotenv import load_dotenv
import os

In [27]:
load_dotenv()
tavily_tool = TavilySearch()
os.environ["LANGCHAIN_PROJECT"] = "langchain_agents_old"
openai_model = init_chat_model(model="gpt-4.1-mini")


In [28]:
tavily_tool.invoke({"query": "What is the weather like in Mumbai?"})

{'query': 'What is the weather like in Mumbai?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.climatestotravel.com/climate/india/mumbai',
   'title': 'Mumbai climate: weather by month, temperature, rain',
   'content': 'In Mumbai, the **average temperature** of the coldest month (January) is of 75.2 °F, that of the warmest month (May) is of 86.4 °F. Image 9: Warm thermometer iconOn the warmest days of the month, the temperature usually reaches around 96 °F. Image 14: Warm thermometer iconOn the warmest days of the month, the temperature usually reaches around 99 °F. Image 19: Warm thermometer iconOn the warmest days of the month, the temperature usually reaches around 102 °F. Image 24: Warm thermometer iconOn the warmest days of the month, the temperature usually reaches around 100 °F. Image 29: Warm thermometer iconOn the warmest days of the month, the temperature usually reaches around 97 °F.',
   'score': 0.73322314,
   'raw_content

In [29]:
tools = [tavily_tool]


In [47]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("hwchase17/openai-functions-agent")

In [49]:
print(prompt[0]) #prompt

prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant') additional_kwargs={}


### MassagePlaceholder

It's used when you need to insert a dynamic list of messages into a prompt template, where the number and content of messages can vary. Common use cases:

* Chat History - Insert conversation history
* Agent Scratchpad - Store intermediate agent reasoning steps
* Multi-turn Conversations - Maintain context across turns

In [32]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Agent Scratchpad

The agent does not call a tool just once to get the desired answer, but has a structure that calls tools repeatedly until the desired answer is obtained. Each time you call a tool, what the previous call was like, information about the previous call, and the result are stored in this field.

In [55]:
# from langgraph.prebuilt import create_react_agent (deprecated)
from langchain.agents import create_agent
from langchain.messages import HumanMessage

# agent = create_react_agent(openai_model, tools) (deprecated)

agent = create_agent(
    model=openai_model, 
    tools=[tavily_tool]
)

In [ ]:
# adding pre-built prompt to the agent is removed and it is now part of LangGraph
agent.invoke({
    "messages": [{ "role": "user", "content": "What is the weather like in Mumbai?"}]
})

{'messages': [HumanMessage(content='What is the weather like in Mumbai?', additional_kwargs={}, response_metadata={}, id='3e411fd0-ecdf-4e4a-acba-aabc41da5d99'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 1276, 'total_tokens': 1295, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_24710c7f06', 'id': 'chatcmpl-CggV8CxzrwgsY9LQrgN2k3ErDuoTP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--d4ffcfd3-f56a-4c65-9e6a-98a682aa1a2e-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'current weather in Mumbai'}, 'id': 'call_UvcFoNfLwLW2FcPvn0LVMzV2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 

In [60]:
agent.invoke({
    "messages": [{ "role": "user", "content": "Hi how are you?"}]
})

{'messages': [HumanMessage(content='Hi how are you?', additional_kwargs={}, response_metadata={}, id='149b7505-421d-42ed-8024-cf74f8779f75'),
  AIMessage(content="Hello! I'm doing well, thank you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 1273, 'total_tokens': 1290, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_24710c7f06', 'id': 'chatcmpl-CggWKafPJm1IKoTDLQaeFFeSn7mQ7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--79244789-6613-4384-b335-4d1de37f1c77-0', usage_metadata={'input_tokens': 1273, 'output_tokens': 17, 'total_tokens': 1290, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_to